Document Classifier


In [1]:
!unzip /content/drive/MyDrive/rap/CV_Dataset.zip

Archive:  /content/drive/MyDrive/rap/CV_Dataset.zip
   creating: Orginal/email/
  inflating: Orginal/email/doc_000042.png  
  inflating: Orginal/email/doc_000076.png  
  inflating: Orginal/email/doc_000079.png  
  inflating: Orginal/email/doc_000111.png  
  inflating: Orginal/email/doc_000133.png  
  inflating: Orginal/email/doc_000142.png  
  inflating: Orginal/email/doc_000148.png  
  inflating: Orginal/email/doc_000165.png  
  inflating: Orginal/email/doc_000195.png  
  inflating: Orginal/email/doc_000196.png  
  inflating: Orginal/email/doc_000238.png  
  inflating: Orginal/email/doc_000255.png  
  inflating: Orginal/email/doc_000260.png  
  inflating: Orginal/email/doc_000275.png  
  inflating: Orginal/email/doc_000278.png  
  inflating: Orginal/email/doc_000279.png  
  inflating: Orginal/email/doc_000282.png  
  inflating: Orginal/email/doc_000297.png  
  inflating: Orginal/email/doc_000333.png  
  inflating: Orginal/email/doc_000347.png  
  inflating: Orginal/email/doc_000363.pn

Downloading and Extracting dataset From Google Drive or Github Zip


Importing Neccessary Packages


In [5]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torchvision.models as models
from transformers import AutoImageProcessor, AutoModelForImageClassification

Downloading and Loading a Pretrained Dit Model 

In [6]:
processor = AutoImageProcessor.from_pretrained("microsoft/dit-base-finetuned-rvlcdip")
model = AutoModelForImageClassification.from_pretrained("microsoft/dit-base-finetuned-rvlcdip")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.9/dist-packages/transformers/models/beit/image_processing_beit.py:111: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [7]:
model

BeitForImageClassification(
  (beit): BeitModel(
    (embeddings): BeitEmbeddings(
      (patch_embeddings): BeitPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BeitEncoder(
      (layer): ModuleList(
        (0): BeitLayer(
          (attention): BeitAttention(
            (attention): BeitSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BeitSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): BeitIntermediate(
            (dense): Linear(in_featur

for FineTuning making the base parameters grad to false. Since it should not update itself in backpropogation

In [8]:
for param in model.base_model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

Initializing the Device

In [9]:
device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
device

device(type='cuda')

Loading the Tensors to the Hardware Device

In [10]:
model = model.to(device)

Creating Custom Dataset to load the Images with its respective labels


In [11]:
class Dataset():
  def __init__(self,folder,train=True):
    self.fold = folder
    self.subfolders = os.listdir(self.fold)
    self.lab = {}
    for idx,ele in enumerate(self.subfolders):
      self.lab[ele] = idx
    self.rlab = {i:k for k,i in self.lab.items()}
    self.img = []
    for i in self.subfolders:
      labimg = os.listdir(os.path.join(self.fold,i))
      trainl = labimg[:int(len(labimg)*0.9)]
      if train == False:
        trainl = labimg[int(len(labimg)*0.9):]
      for j in trainl:
        self.img.append([j,self.lab[i]])
    
  def __len__(self):
    return len(self.img)

  def __getitem__(self,idx):
    img_path = self.img[idx]
    image = Image.open(os.path.join(self.fold,self.rlab[img_path[1]],img_path[0]))
    image = image.convert("RGB")
    image = transforms.Resize(256)(image)
    image = transforms.CenterCrop(224)(image)
    image = transforms.ToTensor()(image)
    image = image.to(device)
    image = image.unsqueeze(0)
    label = img_path[1]
    label = torch.tensor(label,device=device)
    return image,label.unsqueeze(0)

  def label(self,idx):
    return self.rlab[idx]

In [12]:
traindata = Dataset(folder="Orginal",train=True)
len(traindata)

148

Choosing CrossEntroppy loss as loss function 
For optimizer Choosing adam with learning rate 0.01

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [14]:
model.train()
epochs = 10
for i in range(epochs):
  for batch_indx,(x,y) in enumerate(traindata):
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output.logits,y)
    loss.backward()
    optimizer.step()
  print(f'The loss at epoch {i} is {loss:.4f}')


#SKIPPING THIS STEP RIGHT NOW FOR TIME PURPOSE

KeyboardInterrupt: ignored

evaluating the model

In [81]:
model.eval()
t = model(x)
out = t.logits[0]
torch.tensor([out[2],out[6],out[14]]).argmax()

tensor(2)

In [61]:
lb = {}
for i in range(16):
  lb[i] = model.config.id2label[i]
lb

{0: 'letter',
 1: 'form',
 2: 'email',
 3: 'handwritten',
 4: 'advertisement',
 5: 'scientific report',
 6: 'scientific publication',
 7: 'specification',
 8: 'file folder',
 9: 'news article',
 10: 'budget',
 11: 'invoice',
 12: 'presentation',
 13: 'questionnaire',
 14: 'resume',
 15: 'memo'}

In [15]:
traindata.label(0)

'resume'

storing the Model in Drive

In [84]:
torch.save(model, '/content/drive/MyDrive/rap/lcd.pth')

Loading the model

In [85]:
model2 = torch.load('/content/drive/MyDrive/rap/lcd.pth')

In [86]:
model2.eval()

BeitForImageClassification(
  (beit): BeitModel(
    (embeddings): BeitEmbeddings(
      (patch_embeddings): BeitPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BeitEncoder(
      (layer): ModuleList(
        (0): BeitLayer(
          (attention): BeitAttention(
            (attention): BeitSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BeitSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): BeitIntermediate(
            (dense): Linear(in_featur

In [114]:
x,y = traindata[110]

In [115]:
t = model2(x)
out = t.logits[0]
torch.tensor([out[2],out[6],out[14]]).argmax()

tensor(2)

In [116]:
model.config.id2label[14] #2-email,6-project,14-resume

'resume'